# Miembros del grupo "Los hiperparámetros" 🙍‍♂️🙍‍♀️🙍‍♂️

- MIGUEL GONZÁLEZ GARCÍA
- ROSA LÓPEZ ESCALONA
- JAVIER QUESADA PAJARES

---

# MODELO HÍBRIDO PONDERADO: NMF + SVD (Public Score: 1.225)
---

# Imports 🔧

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split

# Preprocesamiento del dataset 🧹

In [3]:
train_data = pd.read_csv("data/train.csv", sep=",")
test_data = pd.read_csv("data/test.csv", sep=",")

global_mean = train_data['rating'].mean()
print(f"Media global de ratings: {global_mean:.4f}")

Media global de ratings: 7.6047


In [4]:
train_data.head(5)

,user,item,rating
0,1,25715,7.0
1,1,25716,10.0
2,5,25851,9.0
3,6,25923,5.0
4,7,25924,6.0


In [5]:
test_data.head(5)

,ID,user,item
0,0,8117,268
1,1,10512,24393
2,2,534,1334
3,3,10984,6550
4,4,9093,22128


In [6]:
#train_data, val_split = train_test_split(train_data, test_size=0.2, random_state=42)

## Creación de matriz dispersa 🧩

In [ ]:
unique_users = train_data['user'].unique()
unique_items = train_data['item'].unique()

user_to_index = {user: idx for idx, user in enumerate(unique_users)}
item_to_index = {item: idx for idx, item in enumerate(unique_items)}

num_users = len(unique_users)
num_items = len(unique_items)

print(f"Número de usuarios: {num_users}")
print(f"Número de ítems: {num_items}")

Número de usuarios: 73456
Número de ítems: 171171


In [ ]:
# Crea la matriz dispersa
rows = train_data['user'].map(user_to_index).values
cols = train_data['item'].map(item_to_index).values
ratings = train_data['rating'].values

R_sparse = csr_matrix((ratings, (rows, cols)), shape=(num_users, num_items))

print(f"Matriz dispersa creada con dimensiones: {R_sparse.shape}")
print("Número de ratings no nulos en la matriz:", R_sparse.nnz)

Matriz dispersa creada con dimensiones: (73456, 171171)
Número de ratings no nulos en la matriz: 390351


# Entrenamiento NMF 🏋️

In [ ]:
# Hiperparámetros
k = 20  # Número de factores latentes
lambda_ = 0.015  # Regularización
learning_rate = 0.003  # Tasa de aprendizaje
num_epochs = 65  # Número de iteraciones

mu = global_mean

# Inicialización aleatoria de las matrices latentes U y V
U = np.random.normal(scale=1./k, size=(num_users, k))
V = np.random.normal(scale=1./k, size=(num_items, k))

# Inicialización de sesgos
bu = np.zeros(num_users)
bi = np.zeros(num_items)

print(f"Factores latentes inicializados: U ({U.shape}), V ({V.shape})")
print(f"Sesgos inicializados: bu ({bu.shape}), bi ({bi.shape})")


Factores latentes inicializados: U ((73456, 20)), V ((171171, 20))
Sesgos inicializados: bu ((73456,)), bi ((171171,))


In [10]:
def train_nmf_with_bias(R_sparse, U, V, bu, bi, mu, lambda_, lr, num_epochs):
    rows, cols = R_sparse.nonzero()  # Índices de las posiciones no nulas en la matriz
    num_ratings = len(rows)

    for epoch in range(num_epochs):
        total_cost = 0
        for idx in range(num_ratings):
            i = rows[idx]  # Usuario
            j = cols[idx]  # Ítem
            r_ij = R_sparse[i, j]  # Rating real del usuario i sobre el ítem j

            # Predicción incluyendo el sesgo
            pred_ij = mu + bu[i] + bi[j] + np.dot(U[i, :], V[j, :])
            error = r_ij - pred_ij

            # Actualización de los factores latentes y sesgos
            bu[i] += lr * (error - lambda_ * bu[i])
            bi[j] += lr * (error - lambda_ * bi[j])
            U[i, :] += lr * (error * V[j, :] - lambda_ * U[i, :])
            V[j, :] += lr * (error * U[i, :] - lambda_ * V[j, :])

            # Costo regularizado
            total_cost += error**2 + (lambda_ / 2) * (np.linalg.norm(U[i, :])**2 + np.linalg.norm(V[j, :])**2 + bu[i]**2 + bi[j]**2)

        print(f"Época {epoch + 1}/{num_epochs} - Costo total: {total_cost:.4f}")

    return U, V, bu, bi


In [ ]:
print("Iniciando entrenamiento NMF...")
U, V, bu, bi = train_nmf_with_bias(R_sparse, U, V, bu, bi, mu, lambda_, learning_rate, num_epochs)
print("Entrenamiento completado.")

Iniciando entrenamiento NMF...
Época 1/65 - Costo total: 1244759.2920
Época 2/65 - Costo total: 1167391.4999
Época 3/65 - Costo total: 1125103.7221
Época 4/65 - Costo total: 1094275.4252
Época 5/65 - Costo total: 1069459.8229
Época 6/65 - Costo total: 1048376.3684
Época 7/65 - Costo total: 1029814.0350
Época 8/65 - Costo total: 1013019.6689
Época 9/65 - Costo total: 997451.3513
Época 10/65 - Costo total: 982662.6250
Época 11/65 - Costo total: 968263.0777
Época 12/65 - Costo total: 953964.5964
Época 13/65 - Costo total: 939719.4864
Época 14/65 - Costo total: 925830.1031
Época 15/65 - Costo total: 912776.4980
Época 16/65 - Costo total: 900805.4472
Época 17/65 - Costo total: 889779.3525
Época 18/65 - Costo total: 879400.7071
Época 19/65 - Costo total: 869418.4923
Época 20/65 - Costo total: 859667.7647
Época 21/65 - Costo total: 850044.6629
Época 22/65 - Costo total: 840483.8425
Época 23/65 - Costo total: 830947.0380
Época 24/65 - Costo total: 821417.2225
Época 25/65 - Costo total: 811893.

# Entrenamiento SVD 🏋️

In [ ]:
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(train_data[['user', 'item', 'rating']], reader)

trainset = data.build_full_trainset()

# División en conjunto de entrenamiento y validación
#trainset, valset = train_test_split(data, test_size=0.2, random_state=42)

# Configuración del modelo SVD
svd = SVD(n_factors=20, n_epochs=50, biased=True, lr_all=0.002, reg_all= 0.02, random_state=42)

print("Entrenando el modelo SVD...")
svd.fit(trainset)
print("Modelo SVD entrenado correctamente.")

Entrenando el modelo SVD...
Modelo SVD entrenado correctamente.


# Predicciones y generación de CSV 🧠

In [ ]:
def predict_rating(uid, iid, U, V, bu, bi, mu):
    try:
        # Predicción usando SVD
        pred_svd = svd.predict(uid, iid).est
    except:
        pred_svd = None

    # Predicción usando NMF con bias
    if uid in user_to_index and iid in item_to_index:
        user_idx = user_to_index[uid]
        item_idx = item_to_index[iid]
        pred_nmf = mu + bu[user_idx] + bi[item_idx] + np.dot(U[user_idx, :], V[item_idx, :])
    else:
        pred_nmf = None

    # Ponderación de las predicciones
    if pred_svd is not None and pred_nmf is not None:
        pred = (0.3 * pred_svd) + (0.7 * pred_nmf)
    elif pred_svd is not None:
        pred = pred_svd
    elif pred_nmf is not None:
        pred = pred_nmf
    else:
        pred = mu  # Si ambos fallan, usar la media global

    return f"{round(max(1, min(10, pred)))}.0"

In [ ]:
print("Generando predicciones finales...")
predictions = []

for _, row in test_data.iterrows():
    uid, iid, row_id = row['user'], row['item'], row['ID']
    pred_rating = predict_rating(uid, iid, U, V, bu, bi, mu)
    predictions.append((row_id, pred_rating))

predictions_df = pd.DataFrame(predictions, columns=["ID", "rating"])

output_filename = "predictions_nmf_svd.csv"
predictions_df.to_csv(output_filename, index=False)
print(f"Archivo '{output_filename}' generado correctamente.")

Generando predicciones finales...
Archivo 'predictions_nmf_svd.csv' generado correctamente.
